In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import mysql.connector
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(1, '../')

import os

In [5]:
def csv_load(ruta_carpeta):
   
    dataframes = []

    for archivo in os.listdir(ruta_carpeta):
        
        if archivo.endswith('.csv'):
            
            path = os.path.join(ruta_carpeta, archivo)

            df = pd.read_csv(path)
            
            dataframes.append((archivo.replace('.csv', ''), df))

    return dataframes

lista_dataframes = csv_load('../data')

names = []
dataframes = []

for nombre, df in lista_dataframes:
    names.append(nombre)
    dataframes.append(df)

In [9]:
# ya tengo las tablas cargadas, falta crear base de datos, crear las tablas y subir los csv a las tablas:

#CONEXION Y CREACION DB:

key = open('password.txt','r').read()
conexion = mysql.connector.connect(user='root', password= key, host='localhost')
 
cursor = conexion.cursor()

cursor.execute('CREATE DATABASE IF NOT EXISTS VitalStatsES')

# SUBIR TABLA ID Y CREAR PRYMARY KEY:

for name in names:

    df = pd.read_csv(f'../data/{name}.csv')
    
    for dato in df.select_dtypes(include='float'):
        df[dato] = pd.to_numeric(df[dato], downcast='float')

    for c in df.select_dtypes('integer'):
        df[dato] = pd.to_numeric(df[dato], downcast='integer')

    if name == 'id':
        
        df.to_sql (name= name,
                    con= conexion,
                    if_exists='append',
                    index=False )

        with cursor.connect() as con:
                con.execute(f'ALTER TABLE `{name}` ADD PRIMARY KEY (`id`);')
                
    else:
        
        df.to_sql (name= name,
                    con= conexion,
                    if_exists='append',
                    index=False )

        with cursor.connect() as con:
                con.execute(f'ALTER TABLE `{name}` ADD FOREIGN KEY (`id`);')
    
    

DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': Not all parameters were used in the SQL statement

In [16]:
key = open('password.txt','r').read()
conexion = mysql.connector.connect(user='root', password= key, host='localhost')
 
cursor = conexion.cursor()

cursor.execute('CREATE DATABASE IF NOT EXISTS VitalStatsES')

for name in names: 
    df = pd.read_csv(f'../data/{name}.csv')
    
    if name == 'id':
        df.to_sql (name = name,
                    con= conexion,
                    if_exists = 'replace',
                    index = False )

DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': Not all parameters were used in the SQL statement